In [ ]:
import numpy as np
import controller_utils
import mujoco
from mujoco import viewer
import tools.rotations as rot

# import rospy

import time
time.sleep(0.4)
# import mediapy as media
%load_ext autoreload
%autoreload 2

In [ ]:
xml_path = 'description/iiwa7_allegro_llm.xml'
obj_name = ''
model = mujoco.MjModel.from_xml_path(xml_path)
data = mujoco.MjData(model)
mujoco.mj_step(model, data)


# viewer.launch(model, data)
view = viewer.launch_passive(model, data)
# in notebook, we need ro run view.sync() manually, and set auto_sync=False

obj_names = ['champagne_1', 'champagne_2', 'champagne_3']
num = 0
obj = obj_names[num]
r = controller_utils.Robot(model, data, view, auto_sync=False, obj_names=obj_names)

q0 = np.array([-0.32032434,  0.02706913, -0.22881953, -1.42621454,  1.3862661 , 0.55966738,  1.79477984 - np.pi*3/2])
r.d.qpos[:7]  = q0
r.step()
view.sync()

In [ ]:
r.modify_joint(q0)
# move the object to a proper pose as an initial pose
p = np.array([0.6,-0.275,1.2])
q = rot.euler2quat([0,0,0])
pose = np.concatenate([p, q])
r.modify_obj_pose(obj, pose)

# move the hand to the grasping configuration

qh = np.array([0, 0.33, 1.05, 1.05,
               0, 0.29, 1.2, 0.6,
               0, 0.32, 1.2, 0.8,
               1, 0.5, 0.71,1])
r.modify_joint(qh)

r.step()
view.sync()

In [ ]:
x0 = np.copy(r.x)
x0[2] += 0.1
qh0 = np.copy(qh)
# F_grasp = [0.7, 0.1, 0.3]
F_grasp = [0.1] # notice that the dimension of F_grasp must be identical to the number of pairs provided in the function `r.pinch_grasp_force`

while True:
    # u_add = r.pinch_grasp_force(F_grasp, pairs=[[0,3], [1,3], [2,3]])
    u_add = r.pinch_grasp_force(F_grasp, pairs=[[0,1]])
    # pinch grasp assumes a grasp formed by a pair of fingertips

    # r.iiwa_hand_go(x0, qh0, kh_scale=[0.2,0.2,0.2,0.2], u_add=u_add)
    r.moveto_attractor(x0, qh0, scaling=1, u_add=u_add)
    view.sync()
    time.sleep(0.002)